## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Kodiak,US,57.79,-152.41,28.40,46,20,8.05,few clouds
1,Hithadhoo,MV,-0.60,73.08,83.26,71,6,4.52,clear sky
2,Tual,ID,-5.67,132.75,83.32,71,30,6.96,scattered clouds
3,Zyryanskoye,RU,56.83,86.63,32.90,96,100,18.45,overcast clouds
4,Oranjemund,NaN,-28.55,16.43,61.20,90,95,10.13,overcast clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_city_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,Hithadhoo,MV,-0.60,73.08,83.26,71,6,4.52,clear sky
2,Tual,ID,-5.67,132.75,83.32,71,30,6.96,scattered clouds
5,Turbat,PK,26.00,63.04,78.15,14,0,4.18,clear sky
8,Kawalu,ID,-7.38,108.21,85.32,64,99,4.92,moderate rain
9,Busselton,AU,-33.65,115.33,78.01,32,0,0.58,clear sky
10,Biak,ID,-0.91,122.88,86.97,57,6,0.47,light rain
11,Kahului,US,20.89,-156.47,86.00,55,40,11.41,scattered clouds
13,Lazaro Cardenas,MX,17.96,-102.20,75.63,74,0,5.10,clear sky
14,Kapaa,US,22.08,-159.32,80.60,83,1,9.17,clear sky
17,Ipixuna,BR,-1.76,-48.80,77.00,94,40,4.70,light rain


In [15]:
# 4a. Determine if there are any empty rows.
preferred_city_df.isnull().sum()

City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_city_df

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,83.26,clear sky,-0.60,73.08,
2,Tual,ID,83.32,scattered clouds,-5.67,132.75,
5,Turbat,PK,78.15,clear sky,26.00,63.04,
8,Kawalu,ID,85.32,moderate rain,-7.38,108.21,
9,Busselton,AU,78.01,clear sky,-33.65,115.33,
10,Biak,ID,86.97,light rain,-0.91,122.88,
11,Kahului,US,86.00,scattered clouds,20.89,-156.47,
13,Lazaro Cardenas,MX,75.63,clear sky,17.96,-102.20,
14,Kapaa,US,80.60,clear sky,22.08,-159.32,
17,Ipixuna,BR,77.00,light rain,-1.76,-48.80,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"] = hotel_df["Hotel Name"].dropna()
hotel_df["Hotel Name"].isnull().sum()

0

In [41]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
hotel_df.to_csv("hotel_df.csv", index_label="City_ID")

In [48]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template =  """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl><dt>Current Weather</dt><dd>{Description}
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [49]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))